### Linear Gaussian Model

In [1]:
import numpy as np 
import math
import scipy.stats
import scipy.io
import sklearn.metrics

#### Auxiliary functions

In [2]:
# Skeleton definition
NUI_SKELETON_POSITION_COUNT = 20

NONE = -1
HIP_CENTER = 0
SPINE = 1
SHOULDER_CENTER = 2
HEAD = 3
SHOULDER_LEFT = 4
ELBOW_LEFT = 5
WRIST_LEFT = 6
HAND_LEFT = 7
SHOULDER_RIGHT = 8
ELBOW_RIGHT = 9
WRIST_RIGHT = 10
HAND_RIGHT = 11
HIP_LEFT = 12
KNEE_LEFT = 13
ANKLE_LEFT = 14
FOOT_LEFT = 15
HIP_RIGHT = 16
KNEE_RIGHT = 17
ANKLE_RIGHT = 18
FOOT_RIGHT = 19

nui_skeleton_names = ( \
    'HIP_CENTER', 'SPINE', 'SHOULDER_CENTER', 'HEAD', \
    'SHOULDER_LEFT', 'ELBOW_LEFT', 'WRIST_LEFT', 'HAND_LEFT', \
    'SHOULDER_RIGHT', 'ELBOW_RIGHT', 'WRIST_RIGHT', 'HAND_RIGHT', \
    'HIP_LEFT', 'KNEE_LEFT', 'ANKLE_LEFT', 'FOOT_LEFT', \
    'HIP_RIGHT', 'KNEE_RIGHT', 'ANKLE_RIGHT', 'FOOT_RIGHT' )

nui_skeleton_conn = ( \
    NONE, \
    HIP_CENTER, \
    SPINE, \
    SHOULDER_CENTER, \
    # Left arm 
    SHOULDER_CENTER, \
    SHOULDER_LEFT,  \
    ELBOW_LEFT,  \
    WRIST_LEFT,  \
    # Right arm 
    SHOULDER_CENTER,  \
    SHOULDER_RIGHT,  \
    ELBOW_RIGHT,  \
    WRIST_RIGHT,  \
    # Left leg 
    HIP_CENTER,  \
    HIP_LEFT,  \
    KNEE_LEFT,  \
    ANKLE_LEFT,  \
    # Right leg 
    HIP_CENTER,  \
    HIP_RIGHT,  \
    KNEE_RIGHT,  \
    ANKLE_RIGHT,  \
)



In [3]:
def load_dataset(file=None):
    """
      Returns the data, the labels and the person id for each action
    """
    import scipy.io
    
    if file is None:
        ex = scipy.io.loadmat('data/data.mat')
    else:
        ex = scipy.io.loadmat(file)
        
    return ex['data'],ex['labels'],ex['individuals']

In [4]:
def log_normpdf(x, mu, sigma):
    """
      Computes the natural logarithm of the normal probability density function
      
    """
    
    probs = np.empty((len(x),1))
    for i in range(len(x)):
        probs[i] =(scipy.stats.norm(mu[i], sigma[i]).logpdf(x[i]))
    
    return probs


In [5]:
def normalize_logprobs(log_probs):
    """
       Returns the log prob normalizes so that when exponenciated
       it adds up to 1 (Useful to normalizes logprobs)
    """
    mm = np.max(log_probs)
    return log_probs - mm - np.log(np.sum(np.exp(log_probs - mm)))


#### Functions to implement

In [6]:
def fit_gaussian(X, W=None):
    """
      Compute the mean and variance of X, 
      You can ignore W for the moment
    """
    mean=np.mean(X)
    #variance=np.std(X)**2
    #variance=np.var(X)
    variance=np.std(X)
    
    return (mean, variance)


In [7]:
def my_cov(x,y,w):
    """
      Useful function for fit_linear_gaussian
    """
    return np.sum(w*x*y)/np.sum(w)-np.sum(w*x)*np.sum(w*y)/np.sum(w)/np.sum(w)


In [8]:
def fit_linear_gaussian(Y,X,W = None):
    """
    Input:
      Y: vector of size D with the observations for the variable
      X: matrix DxV with the observations for the parent variables
                 of X. V is the number of parent variables
      W: vector of size D with the weights of the instances (ignore for the moment)
      
    Outout:
       The betas and sigma
    """
    # row = D = number of observations
    # col = V = number of parents
    [row,col] = X.shape
    
    A = np.ones((col+1, col+1))
    B = np.empty((col+1, 1))
    
    B[0] = np.mean(Y[:])

    for k in range (col):
        A[0,k+1] = np.mean(X[:,k])
        A[k+1,0] = np.mean(X[:,k])
        B[k+1] = np.mean(Y[:]*X[:,k])

    for i in range (col):
        for j in range (col):
            A[i+1,j+1] = np.mean(X[:,i]*X[:,j])
    
    betas = np.linalg.solve(A,B)
    temp = 0
    
    for i in range (col):
        for j in range (col):
            temp = temp + betas[i+1]*betas[j+1]*(np.mean(X[:,i]*X[:,j]) - np.mean(X[:,i])*np.mean(X[:,j]))

    sigma = math.sqrt(np.mean(Y[:]*Y[:]) - np.mean(Y[:])*np.mean(Y[:]) - temp)

    

    return (betas,sigma)

In [9]:
class Model:
    
    def __init__(model, dataset, labels, G):
        model.connectivity = G
        # initialize
        model.class_priors = []
        model.jointparts = [] 
        # fill
        model.set_class_priors(labels)
        model.set_jointparts(dataset, labels, G)
    
    def set_class_priors(model, labels):
        nb_classes=len(np.unique(labels))
        for class_nb in range(nb_classes):
            model.class_priors.append(len(np.where(labels==class_nb))/len(labels))
        
    def set_jointparts(model, dataset, labels, G):
        [r, c, d]=dataset.shape
        # Create a list of joints
        for joint in range(r):
            model.jointparts.append(JointPart(dataset, labels, G, joint))
            
    def isNaiveBayes(model):
        if model.connectivity==None:
            return True
        else:
            return False
              
    

class JointPart:
    

    def __init__(jointpart, dataset, labels, G, joint):
        # initiaization
        jointpart.joint_nb=joint
        jointpart.means = []
        jointpart.betas = []
        jointpart.sigma = []
        # fill depending on G
        if G==None :
            jointpart.fill_Naive_Bayes(dataset[jointpart.joint_nb, :, :], labels)
        else:
            jointpart.fill_Linear_Gaussian(dataset, labels, G)
    
    def fill_Naive_Bayes(jointpart, joint_dataset, labels):
        [r, c]=joint_dataset.shape
        
        # Get all the different the classes, ordered : [1, 2, 3, 8]
        classes=np.unique(labels)
        
        # Get the number of classes
        nb_classes=len(classes)
        
        # Initialize means and sigma
        jointpart.means = np.zeros((3, nb_classes))
        jointpart.sigma = np.zeros((3, nb_classes))
        
        for pos in range(r): # x, y, z
            for class_nb in range(nb_classes): # 1, 2, 3, 8
                # Create mask of index of instances belonging to the class
                mask_class=(labels==classes[class_nb])
                mask_class=np.reshape(mask_class, (1, -1))
                mask_class=mask_class.tolist()[0]
                # get mean and variance
                m, v = fit_gaussian(joint_dataset[pos, mask_class], W=None)
                jointpart.means[pos, class_nb]=m
                jointpart.sigma[pos, class_nb]=v
      
  
    def fill_Linear_Gaussian(jointpart, dataset, labels, G):
        
        # Get all the different the classes, ordered : [1, 2, 3, 8]
        classes=np.unique(labels)
        
        # Get the number of classes
        nb_classes=len(classes)
        
        # Initialize betas and sigma
        jointpart.betas = np.zeros((12, nb_classes))
        jointpart.sigma = np.zeros((3, nb_classes))
        
        # dataset for that specific joint
        joint_dataset=dataset[jointpart.joint_nb, :, :]
        
        [r, c]=joint_dataset.shape
        
        has_parent=True
        
        parent=G[jointpart.joint_nb]
        
        if parent!=-1:
            parent_dataset=dataset[parent, :, :]
        else:
            has_parent=False
        
        for pos in range(r): # x, y, z
            
            for class_nb in range(nb_classes): # 1, 2, 3, 8
                
                # Create mask of index of instances belonging to the class
                    mask_class=(labels==classes[class_nb])
                    mask_class=np.reshape(mask_class, (1, -1))
                    mask_class=mask_class.tolist()[0]
                
                    if has_parent:

                        # get mean and variance
                        b, s = fit_linear_gaussian(joint_dataset[pos, mask_class],np.transpose(parent_dataset[:, mask_class]), W = None)    
                        b=np.reshape(b, (1, -1))
                        b=b.tolist()[0]
                        jointpart.betas[pos*4:pos*4+4, class_nb]=b
                        jointpart.sigma[pos, class_nb]=s

                    else:

                        # if not, we fill betas with Naïve Bayes : fit_gaussian
                        m, v = fit_gaussian(joint_dataset[pos, mask_class], W=None)
                        jointpart.betas[pos*4:pos*4+4, class_nb]=m
                        jointpart.sigma[pos, class_nb]=v
 

    def isNaiveBayes(self):
        if len(jointparts.betas)==0:
            return True
        else:
            return False
        
   
    

In [10]:
def learn_model(dataset, labels, G=None):
    """
    Input:
     dataset: The data as it is loaded from load_data
     labels:  The labels as loaded from load_data
     Graph:   (optional) If None, this def should compute the naive 
           bayes model. If it contains a skel description (pe 
           nui_skeleton_conn, as obtained from skel_model) then it should
           compute the model using the Linear Gausian Model

    Output: the model
     a (tentative) structure for the output model is:
       model.connectivity: the input Graph variable should be stored here 
                           for later use.
       model.class_priors: containing a vector with the prior estimations
                           for each class
       model.jointparts[i] contains the estimated parameters for the i-th joint

          For joints that only depend on the class model.jointparts(i) has:
            model.jointparts(i).means: a matrix of 3 x #classes with the
                   estimated means for each of the x,y,z variables of the 
                   i-th joint and for each class.
            model.jointparts(i).sigma: a matrix of 3 x #classes with the
                   estimated stadar deviations for each of the x,y,z 
                   variables of the i-th joint and for each class.

          For joints that follow a gausian linear model model.jointparts(i) has:
            model.jointparts(i).betas: a matrix of 12 x #classes with the
                   estimated betas for each x,y,z variables (12 in total) 
                   of the i-th joint and for each class label.
            model.jointparts(i).sigma: as above

    """
    
    
    # Everything is set in the class
    
    model=Model(dataset, labels, G)
    
    return model
    
    

In [11]:
def classify_instances(instances, model):
    """    
    Input
       instance: a 20x3x#instances matrix defining body positions of
                 instances
       model: as the output of learn_model

    Output
       probs: a matrix of #instances x #classes with the probability of each
              instance of belonging to each of the classes

    Important: to avoid underflow numerical issues this computations should
               be performed in log space
    """
    
    classes_num = len(model.class_priors) 
    
    if len(instances.shape)==2:
        joints_num, coordinates_num = instances.shape
        instances_num=1  
    else:
        joints_num, coordinates_num, instances_num= instances.shape
        
    
    probs = np.zeros((instances_num, classes_num))
    
    if instances_num==1:
        probs = compute_logprobs(instances,model)
    else:    
        for instance in range(instances_num):
            probs[instance,:] = compute_logprobs(instances[:,:,instance],model)

    return np.exp(probs)




In [12]:
def compute_logprobs(instance, model):
    
    """

       Input
           instance: a 20x3 matrix defining body positions of one instance
           model: as given by learn_model

       Output
           l: a vector of len #classes containing the loglikelihhod of the 
              instance

    """
    
    joints_num, coordinates_num = instance.shape #here we get the 20*3 matrix of one instance
    classes_num = len(model.class_priors) #here we get the number of classes in the model
    logprobs = np.empty((classes_num,1))
        
        
    if model.connectivity == None:

        for each_class in range(classes_num):
            product = math.log(model.class_priors[each_class]) # prior
            for each_joint in range(joints_num): #sum up for all the joints
                # sum up for three coordinates for each joint
                product += sum(log_normpdf(instance[each_joint,:], model.jointparts[each_joint].means[:, each_class], model.jointparts[each_joint].sigma[:, each_class])) 
            logprobs[each_class] = product

        logprobs = list(normalize_logprobs(logprobs) )


    else:
        for each_class in range(classes_num):
            
            product = math.log(model.class_priors[each_class]) # prior
            
            for each_joint in range(joints_num): #sum up for all the joints
                means = []

                for each_coordinate in range(coordinates_num):# mu = beta0 + beta1*parent1 +beta2*parent2 + beta3+parent3 
                    
                    if model.connectivity[each_joint] == -1:
                        means.append(model.jointparts[each_joint].betas[each_coordinate*4,each_class])
                                                            
                    else :
                        temp_mean = model.jointparts[each_joint].betas[each_coordinate*4,each_class] + model.jointparts[each_joint].betas[each_coordinate*4+1,each_class]*instance[model.connectivity[each_joint],0] + model.jointparts[each_joint].betas[each_coordinate*4+2,each_class]*instance[model.connectivity[each_joint],1] + model.jointparts[each_joint].betas[each_coordinate*4+3,each_class]*instance[model.connectivity[each_joint],2]
                        means.append(temp_mean)
  
                product += sum(log_normpdf(instance[each_joint,:], means, model.jointparts[each_joint].sigma[:, each_class]))  #sum for all the 20 joints 

            logprobs[each_class] = product #the probability of a specific class
        
    
    logprobs = list(normalize_logprobs(logprobs))

    
    return logprobs


In [13]:
def generate_random_lgm_samples(n, betas, sigma):
    """Function to generate random samples for a 
       Linear Gaussian Model
       Input:
           n: Number of samples
           betas: vector with the values the the betas from 0 to k
           sigma: standard deviation
    """
    X = np.random.randn(n,betas.shape[0]-1)
    Y = np.random.randn(n)*sigma + np.sum(X*betas[1:],axis=1)+betas[0]
    return X,Y

In [14]:
def predict_label(classified_instances, possible_classes):
    label_pred=np.argmax(classified_instances, axis=1)
    #label_pred=np.reshape(label_pred, (-1, 1))
    for i in range (len(label_pred)):
        label_pred[i]=possible_classes[label_pred[i]]
    return label_pred
    
    

In [15]:
def accuracy(y_pred, y_true):
    y_pred=np.reshape(y_pred, y_true.shape)
    return np.sum(y_pred == y_true)/len(y_true)

## TEST FIT_LINEAR_GAUSSIAN

In [16]:
betas = np.array([1,2,10,4,5,6])
sigma = 0.1
n=100
X,Y=generate_random_lgm_samples(n,betas,sigma)
# This following call should output  betas and sigma close to the above ones
fit_linear_gaussian(Y,X)

(array([[1.00240848],
        [1.99273976],
        [9.97807045],
        [3.99790738],
        [4.98971808],
        [6.01638476]]),
 0.10179678344885083)

# RUNNING 

In [16]:
data, labels, individuals = load_dataset(file=None)

In [17]:
possible_classes=np.unique(labels)

## TEST ON VALIDATION DATA

In [21]:
### TEST 2 ###

d = scipy.io.loadmat('data/validation_data.mat')

data_small        = d['data_small']
labels_small      = d['labels_small'].squeeze()
individuals_small = d['individuals_small'].squeeze()
#train_indexes     = np.array(d['train_indexes'].squeeze(),dtype=bool)
#test_indexes      = np.array(d['test_indexes'].squeeze(),dtype=bool)
#model_nb          = d['model_nb']
#model_lg          = d['model_lg']
#accur_lg          = d['accur_lg']
#accur_nb          = d['accur_nb']


### TEST ON VALIDATION DATA : Naïve Bayes

In [20]:
# Params of following model should be similar to those of model_nb above
naive_bayes_model = learn_model(data_small[:,:,train_indexes],labels_small[train_indexes])

# Params of following model should be similar to those of model_lg above
#  
#  Note1: The implementation used to get model_lg stored the variance
#  Note2: The implementation used to get model_lg added a epsilon to the 
#         variance to avoid numerical issues. Specifically the code at 
#         the end of fit_linear_gaussian used is:
#
#    sigma = np.sqrt(sigma)                                                       
#                                                                                 
#    if sigma == 0 or type(sigma) == 'complex':                                   
#        sigma = .01                                                              
#    else:                                                                        
#        sigma = sigma + .01                                                      
#                                                                                 
#    sigma = sigma**2                                                             
#                                                                                 
#    return (betas,sigma) 
#

# The accuracy of naive_bayes_model on data_small[:,:,test_indexes] that have labels
# labels_small[test_indexes] should be aprox accur_nb
nb_prob = classify_instances(data_small[:,:,test_indexes], naive_bayes_model)


In [21]:
y_pred_nb=predict_label(nb_prob, possible_classes)
print(accuracy(y_pred_nb, labels_small[test_indexes]))

0.95


In [22]:
print(accur_nb)

[[0.95]]


In [23]:
print(naive_bayes_model.jointparts[0].means)

[[ 0.0165528  -0.02337965 -0.0247925  -0.08551245]
 [ 0.18375145 -0.4245773   0.12681275  0.1400868 ]
 [ 2.83720545  2.86587695  2.840341    2.6486826 ]]


In [24]:
print(model_nb[0, 0][0][0][0][0])

[[ 0.0165528  -0.02337965 -0.0247925  -0.08551245]
 [ 0.18375145 -0.4245773   0.12681275  0.1400868 ]
 [ 2.83720545  2.86587695  2.840341    2.6486826 ]]


Test if the we get the same means and sigmas as model_nb

In [25]:
c=0

#check that our Naïve Bayes and model_nb are equivalent (compare means and sigma)
for i in range(20):
    if (naive_bayes_model.jointparts[i].means-model_nb[0, 0][0][0][i][0]>1e-10).any() or (naive_bayes_model.jointparts[i].sigma-model_nb[0, 0][0][0][i][1]>1e-10).any():
        c=c+1

if c:
    print("Naïve Bayes Model is not validated")
else:
    print("Naïve Bayes Model is validated")
    

Naïve Bayes Model is validated


We get the same means and sigmas

### TEST ON VALIDATION DATA : Linear Gaussian

In [26]:
linear_gaussian_model = learn_model(data_small[:,:,train_indexes],labels_small[train_indexes], nui_skeleton_conn)

# The accuracy of linear_gaussian_model on data_small[:,:,test_indexes] that have labels
# labels_small[test_indexes] should be aprox accur_lg
lg_prob = classify_instances(data_small[:,:,test_indexes], linear_gaussian_model)

In [27]:
y_pred_lg=predict_label(lg_prob, possible_classes)
print(accuracy(y_pred_lg, labels_small[test_indexes]))

1.0


In [28]:
print(accur_lg)

[[0.975]]


Test if the we get the same betas and sigmas as model_lg

In [29]:
c=0

#check that our Linear Gaussian Model and model_lg are equivalent (compare means and sigma) (1e-10)
for i in range(2,20):
    if (linear_gaussian_model.jointparts[i].betas-model_lg[0, 0][0][0][i][0]>1e-10).any() or (linear_gaussian_model.jointparts[i].sigma-model_lg[0, 0][0][0][i][1]>1e-10).any():
        c=c+1

if c:
    print("Linear Gaussian Model is not validated")
else:
    print("Linear Gaussian Model is validated")
    



Linear Gaussian Model is validated


We get the same betas and sigmas

# TESTING MODELS

## TESTING MODELS on the whole dataset

### TESTING MODELS on the whole dataset : Naïve Bayes

In [30]:
nb_model=learn_model(data, labels, G=None)
predictions_probs=classify_instances(data, nb_model)
y_pred=predict_label(predictions_probs, possible_classes)

In [31]:
accuracy(y_pred, labels)

0.9525672371638142

In [32]:
for i in range (len(y_pred)):  
    if y_pred[i]!=labels[i]:
        print ("i=", i, ", y_pred=", y_pred[i], " and y_true=", labels[i])

i= 92 , y_pred= 8  and y_true= [1]
i= 93 , y_pred= 8  and y_true= [1]
i= 94 , y_pred= 8  and y_true= [1]
i= 95 , y_pred= 8  and y_true= [1]
i= 96 , y_pred= 8  and y_true= [1]
i= 97 , y_pred= 8  and y_true= [1]
i= 98 , y_pred= 8  and y_true= [1]
i= 99 , y_pred= 8  and y_true= [1]
i= 100 , y_pred= 8  and y_true= [1]
i= 101 , y_pred= 8  and y_true= [1]
i= 408 , y_pred= 3  and y_true= [1]
i= 409 , y_pred= 3  and y_true= [1]
i= 426 , y_pred= 8  and y_true= [1]
i= 428 , y_pred= 8  and y_true= [1]
i= 429 , y_pred= 8  and y_true= [1]
i= 430 , y_pred= 8  and y_true= [1]
i= 434 , y_pred= 8  and y_true= [1]
i= 435 , y_pred= 8  and y_true= [1]
i= 446 , y_pred= 8  and y_true= [1]
i= 447 , y_pred= 8  and y_true= [1]
i= 448 , y_pred= 8  and y_true= [1]
i= 449 , y_pred= 8  and y_true= [1]
i= 450 , y_pred= 8  and y_true= [1]
i= 451 , y_pred= 8  and y_true= [1]
i= 452 , y_pred= 8  and y_true= [1]
i= 453 , y_pred= 8  and y_true= [1]
i= 454 , y_pred= 8  and y_true= [1]
i= 480 , y_pred= 3  and y_true= [1]


In [33]:
sklearn.metrics.confusion_matrix(labels, y_pred)

array([[480,   0,   3,  25],
       [  0, 500,   0,   0],
       [  0,   0, 474,  48],
       [  0,   0,  21, 494]])

### TESTING MODELS on the whole dataset : Linear Gaussian

In [34]:
lg_model=learn_model(data, labels, nui_skeleton_conn)
lg_predictions_probs=classify_instances(data, lg_model)
lg_y_pred=predict_label(lg_predictions_probs, possible_classes)

In [35]:
accuracy(lg_y_pred, labels)

0.9921760391198045

In [36]:
sklearn.metrics.confusion_matrix(labels, lg_y_pred)

array([[493,   0,   0,  15],
       [  0, 500,   0,   0],
       [  0,   0, 521,   1],
       [  0,   0,   0, 515]])

# TRAIN and VALIDATION datasets

In [37]:
NB_prob_val=classify_instances(data_small, nb_model)
NB_y_pred_val=predict_label(NB_prob_val, possible_classes)
accuracy(NB_y_pred_val, labels_small)

0.9583333333333334

In [38]:
sklearn.metrics.confusion_matrix(labels_small, NB_y_pred_val)

array([[29,  0,  0,  1],
       [ 0, 30,  0,  0],
       [ 0,  0, 28,  2],
       [ 0,  0,  2, 28]])

In [39]:
LG_prob_val=classify_instances(data_small, lg_model)
LG_y_pred_val=predict_label(LG_prob_val, possible_classes)
accuracy(LG_y_pred_val, labels_small)

1.0

In [40]:
sklearn.metrics.confusion_matrix(labels_small, LG_y_pred_val)

array([[30,  0,  0,  0],
       [ 0, 30,  0,  0],
       [ 0,  0, 30,  0],
       [ 0,  0,  0, 30]])

# TRAIN/TEST SET


Split the data into a training and a test set

In [48]:
indices=np.arange(len(labels))
training_indices=indices%3>0
test_indices=indices%3==0

training_data=data[:,:,training_indices]
training_labels=labels[training_indices]
test_data=data[:,:,test_indices]
test_labels=labels[test_indices]

In [49]:
print(training_data.shape)
print(len(training_labels))
print(test_data.shape)
print(len(test_labels))

(20, 3, 1363)
1363
(20, 3, 682)
682


In [50]:
model_NB=learn_model(training_data, training_labels, G=None)
NB_predictions_probs=classify_instances(test_data, model_NB)
y_pred_NB=predict_label(NB_predictions_probs, possible_classes)
accuracy(y_pred_NB, test_labels)

0.9472140762463344

In [51]:
sklearn.metrics.confusion_matrix(test_labels, y_pred_NB)

array([[158,   0,   2,  10],
       [  0, 166,   0,   0],
       [  0,   0, 157,  17],
       [  0,   0,   7, 165]])

In [52]:
model_LG=learn_model(training_data, training_labels, nui_skeleton_conn)
LG_predictions_probs=classify_instances(test_data, model_LG)
y_pred_LG=predict_label(LG_predictions_probs, possible_classes)
accuracy(y_pred_LG, test_labels)

0.9868035190615836

In [53]:
sklearn.metrics.confusion_matrix(test_labels, y_pred_LG)

array([[162,   0,   0,   8],
       [  0, 166,   0,   0],
       [  0,   0, 173,   1],
       [  0,   0,   0, 172]])

## CROSS VALIDATION

In [18]:
def cross_validation(data, labels, k, G):
    nb_instances=len(labels)
    k_percent=math.floor(nb_instances/k)
    indices=np.arange(nb_instances)
    np.random.shuffle(indices)
    shuffled_indices =  indices
    possible_classes=np.unique(labels)
    
    models=[]
    accuracies=[]

    for i in range(10):
        print(i)
        training_indices=shuffled_indices[k_percent*i:k_percent*i+k_percent]
        test_indices=np.concatenate((shuffled_indices[:k_percent*i], shuffled_indices[k_percent*i+k_percent:]), axis=None)
        #test_indices=shuffled_indices[:k_percent*i].append(shuffled_indices[k_percent*i+k_percent:])
        training_data=data[:,:,training_indices]
        training_labels=labels[training_indices]
        test_data=data[:,:,test_indices]
        test_labels=labels[test_indices]
        
        models.append(learn_model(training_data, training_labels, G))
        prob=classify_instances(test_data, models[i])
        y_pred=predict_label(prob, possible_classes)
        accuracies.append(accuracy(y_pred, test_labels))
        print("accuracy : ",accuracies[i])
        
    
    return models, accuracies
    
    

In [19]:
models_nb, accuracies_nb=cross_validation(data, labels, 10, G=None)

0
accuracy :  0.9750135795763172
1
accuracy :  0.9440521455730582
2
accuracy :  0.9272134709397066
3
accuracy :  0.9359043997827268
4
accuracy :  0.9755567626290059
5
accuracy :  0.9304725692558392
6
accuracy :  0.9293862031504617
7
accuracy :  0.9429657794676806
8
accuracy :  0.9424225964149918
9
accuracy :  0.9565453557848995


In [20]:
models_lg, accuracies_lg=cross_validation(data, labels, 10, nui_skeleton_conn)

0
accuracy :  0.9891363389462249
1
accuracy :  0.9885931558935361
2
accuracy :  0.9918522542096686
3
accuracy :  0.9853340575774036
4
accuracy :  0.9815317762085823
5
accuracy :  0.9929386203150462
6
accuracy :  0.9831613253666486
7
accuracy :  0.9842476914720261
8
accuracy :  0.986420423682781
9
accuracy :  0.9853340575774036


In [22]:
#testing the best NB model

model_nb=models_nb[np.argmax(accuracies_nb)]
prob_nb_val=classify_instances(data_small, model_nb)
y_pred_nb_val=predict_label(prob_nb_val, np.unique(labels))
accuracy(y_pred_nb_val, labels_small)

0.9833333333333333

In [23]:
sklearn.metrics.confusion_matrix(labels_small, y_pred_nb_val)

array([[29,  0,  0,  1],
       [ 0, 30,  0,  0],
       [ 0,  0, 30,  0],
       [ 0,  0,  1, 29]])

In [24]:
#testing the best  model

model_lg=models_lg[np.argmax(accuracies_lg)]
prob_lg_val=classify_instances(data_small, model_lg)
y_pred_lg_val=predict_label(prob_lg_val, np.unique(labels))
accuracy(y_pred_lg_val, labels_small)

1.0

In [25]:
sklearn.metrics.confusion_matrix(labels_small, y_pred_lg_val)

array([[30,  0,  0,  0],
       [ 0, 30,  0,  0],
       [ 0,  0, 30,  0],
       [ 0,  0,  0, 30]])

In [26]:
PNB=classify_instances(data, model_nb)
YPREDNB=predict_label(PNB, np.unique(labels))
accuracy(YPREDNB, labels)

0.9770171149144254

In [27]:
sklearn.metrics.confusion_matrix(labels, YPREDNB)

array([[479,   0,   4,  25],
       [  0, 500,   0,   0],
       [  0,   0, 516,   6],
       [  0,   0,  12, 503]])

In [28]:
PLG=classify_instances(data, model_lg)
YPREDLG=predict_label(PLG, np.unique(labels))
accuracy(YPREDLG, labels)

0.993643031784841

In [29]:
sklearn.metrics.confusion_matrix(labels, YPREDLG)

array([[501,   0,   0,   7],
       [  0, 500,   0,   0],
       [  0,   0, 520,   2],
       [  0,   0,   4, 511]])